In [9]:
from nn import NeuralNet
import panel as pn
import pandas as pd
import torch
import matplotlib.pyplot as plt
from torchvision.transforms import functional as F
from matplotlib.image import imread
from io import BytesIO
from PIL import Image
import os
pn.extension()

In [38]:
class NeuralNet(torch.nn.Module):
    def __init__(self, input_size, hidden_size):
        super(NeuralNet, self).__init__()
        # Define the layers
        self.l1 = torch.nn.Linear(input_size, hidden_size)  # Input layer to hidden layer
        self.l2 = torch.nn.Linear(hidden_size, 1)  # Hidden layer to output layer (binary classification)
    
    def forward(self, x):
        # Apply the first layer and ReLU activation function
        x = torch.relu(self.l1(x))
        # Apply the second layer (no activation needed for binary output)
        x = self.l2(x)
        return x

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Path to the saved model weights
model_path = "./data/model.pth"

# Load the dataset and preprocess
raw_data = pd.read_csv("balanced.csv")  # Update this path if needed

# Normalize the dataset (excluding the "status" column)
processed_data = (raw_data.iloc[:, 1:] - raw_data.iloc[:, 1:].mean()) / (raw_data.iloc[:, 1:].std() + 1e-8)

# Check the number of features in the processed data
print(f"Number of features in processed data: {processed_data.shape[1]}")

# Dynamically set input size based on the number of features
#input_size = processed_data.shape[1]  # Number of columns in the dataset (excluding status column)
hidden_size = 20  # Hidden layer size can be adjusted as needed

# Initialize the model dynamically based on input size
model = NeuralNet(input_size=108, hidden_size=hidden_size).to(device) #108 columns to be processed

# Load the saved model weights
try:
    model.load_state_dict(torch.load(model_path, map_location=device))
    print("Model weights loaded successfully!")
except FileNotFoundError:
    print("No pre-trained model found. Please ensure the correct path or train the model first.")
    # Optional: Initialize weights if the file is not found
    model.apply(torch.nn.init.xavier_uniform_)

Number of features in processed data: 108
Model weights loaded successfully!


C:\Users\Anuraag (University)\AppData\Local\Temp\ipykernel_40476\1437049497.py:39: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path,

In [36]:
# Set the model to evaluation mode
model.eval()

# Save the model weights (after training if needed)
torch.save(model.state_dict(), model_path)

# Function to make predictions dynamically
def predict(sensor_values):
    input_tensor = torch.tensor([sensor_values], dtype=torch.float).to(device)
    output = model(input_tensor)

    # Print the raw output from the model before sigmoid
    print(f"Raw output: {output}")

    probability = torch.sigmoid(output).item()
    prediction = "OK" if probability > 0.5 else "NOK"
    return prediction, probability
    
# Create widgets for dynamically adjusting sensor values
sensor_widgets = {
    col: pn.widgets.FloatSlider(name=col, start=-1, end=1, step=0.01, value=0)
    for col in processed_data.columns[1:]  # Skip the "status" column
}

# Create a slider for the status (0 or 1)
status_slider = pn.widgets.IntSlider(name="Status", start=0, end=1, step=1, value=0)

# Update predictions when user interacts with widgets
def update_prediction(event=None):
    sensor_values = [widget.value for widget in sensor_widgets.values()]
    prediction, probability = predict(sensor_values)
    prediction_text.value = f"Prediction: **{prediction}** (Probability: {probability:.2f})"

    # Update the status slider based on the prediction
    if prediction == "OK":
        status_slider.value = 1
    else:
        status_slider.value = 0

# Organize sensor widgets in a grid layout (adjust number of columns based on your preference)
sensor_grid = pn.GridBox(*sensor_widgets.values(), ncols=3, sizing_mode='stretch_width')

# Prediction output text
prediction_text = pn.pane.Markdown("Prediction: **--** (Probability: --)")

# Button to trigger prediction update
update_button = pn.widgets.Button(name="Update Prediction", button_type="primary")
update_button.on_click(update_prediction)

# Visualize raw vs processed data for comparison
def plot_graphs():
    fig, axes = plt.subplots(1, 2, figsize=(10, 5))  # Adjust size for better fitting
    # Plot raw data
    axes[0].plot(raw_data.iloc[:, 2], label="Raw Data", color='blue')
    axes[0].set_title("Raw Data")
    axes[0].legend()
    
    # Plot processed data
    axes[1].plot(processed_data.iloc[:, 1], label="Processed Data", color='green')
    axes[1].set_title("Processed Data")
    axes[1].legend()
    
    plt.tight_layout()
    return fig

# Generate the figure first and pass it to Matplotlib pane
fig = plot_graphs()  # Call the function and get the figure
graphs_panel = pn.pane.Matplotlib(fig, tight=True, width=500)  # Resize the graph to fit better

# Display generated images in a grid layout
def display_images(image_folder='./images'):
    image_panes = []
    for file in os.listdir(image_folder):
        if file.endswith(".png"):
            image_panes.append(pn.pane.PNG(os.path.join(image_folder, file), width=200))  # Resize images
    return pn.GridBox(*image_panes, ncols=3, sizing_mode='stretch_width')

# Build the interactive dashboard
dashboard = pn.Column(
    pn.Row(pn.Column("<h2>Interactive CNN Dashboard</h2>"), align="center"),
    pn.Row(sensor_grid, pn.Column(prediction_text, update_button), sizing_mode='stretch_both'),
    pn.Row(status_slider),  # Add the status slider to the dashboard
    pn.Row("<h3>Graph Comparison</h3>", graphs_panel),
    pn.Row("<h3>Generated Images</h3>", display_images())
)

# Serve the dashboard locally
dashboard.show(port=8116)  # Opens the dashboard in the browser

Launching server at http://localhost:8116


ERROR:bokeh.server.protocol_handler:error handling message
 message: Message 'PATCH-DOC' content: {'events': [{'kind': 'MessageSent', 'msg_type': 'bokeh_event', 'msg_data': {'type': 'event', 'name': 'button_click', 'values': {'type': 'map', 'entries': [['model', {'id': 'b4c59756-27b5-47d5-bd72-0e09e2fae411'}]]}}}]} 
 error: RuntimeError('mat1 and mat2 shapes cannot be multiplied (1x107 and 108x20)')
Traceback (most recent call last):
  File "c:\Python\Lib\site-packages\bokeh\server\protocol_handler.py", line 94, in handle
    work = await handler(message, connection)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Python\Lib\site-packages\bokeh\server\session.py", line 94, in _needs_document_lock_wrapper
    result = func(self, *args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Python\Lib\site-packages\bokeh\server\session.py", line 286, in _handle_patch
    message.apply_to_document(self.document, self)
  File "c:\Python\Lib\site-packages\bokeh\protocol\mess